- simple model using keras frontend, tensorflow backend on python 3

### Libraries

In [3]:
import numpy as np
import pandas as pd
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization

Using TensorFlow backend.


In [4]:
from keras.layers import Input, ELU, Reshape, Concatenate, Activation, Flatten, Dense, Dropout
from keras.models import Model
from keras.losses import categorical_crossentropy
from keras.optimizers import Adadelta
from keras.callbacks import TensorBoard
import keras.backend as K

### Data

In [9]:
train_input = pd.read_csv('../data/data_train.csv')
train_labels = pd.get_dummies(train_input['labels']).as_matrix()
del train_input['labels']
train_input = train_input.as_matrix()
train_input.shape

(33600, 784)

In [10]:
test_input = pd.read_csv("../data/data_val.csv")
test_labels = pd.get_dummies(test_input['labels']).as_matrix()
del test_input['labels']
test_input = test_input.as_matrix()
test_input.shape

(8400, 784)

#### Setting the configuration of the model

In [11]:
num_classes = 10
image_height = 28
image_width = 28
num_channels = 1

batch_size = 128
epochs = 7

## Model

In [13]:
K.clear_session()

In [14]:
x = Input(shape=(image_height*image_width,), name='input')
input_reshaped = Reshape((image_height,image_width,num_channels), input_shape=(image_height*image_width,), name='input_reshape')(x)
conv1 = Conv2D(name="Conv1", filters=5, kernel_size=(3,3), strides=(1,1), activation="relu")(input_reshaped)
conv2 = Conv2D(name="Conv2", filters=10, kernel_size=(5,5), strides=(1,1), activation="relu")(conv1)
conv3 = Conv2D(name="Conv3", filters=15, kernel_size=(5,5), strides=(2,2), activation="relu")(conv2)
conv4 = Conv2D(name="Conv4", filters=20, kernel_size=(7,7), strides=(2,2), activation="relu")(conv3)
convolved_features = Flatten(name="Flat1")(conv4)
dense1 = Dense(400, activation="softmax", name="Dense1")(convolved_features)
dense2 = Dense(num_classes, activation="softmax", name="Dense2")(dense1)

TensorShape([Dimension(None), Dimension(10)])

In [15]:
model = Model(inputs=x, outputs=dense2)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 784)               0         
_________________________________________________________________
input_reshape (Reshape)      (None, 28, 28, 1)         0         
_________________________________________________________________
Conv1 (Conv2D)               (None, 26, 26, 5)         50        
_________________________________________________________________
Conv2 (Conv2D)               (None, 22, 22, 10)        1260      
_________________________________________________________________
Conv3 (Conv2D)               (None, 9, 9, 15)          3765      
_________________________________________________________________
Conv4 (Conv2D)               (None, 2, 2, 20)          14720     
_________________________________________________________________
Flat1 (Flatten)              (None, 80)                0         
__________

In [61]:
tensorboard = TensorBoard(log_dir='./tensorboard_logs/', histogram_freq=1, batch_size=batch_size, write_graph=True, 
                          write_grads=False, write_images=False, embeddings_freq=0, 
                          embeddings_layer_names=None, embeddings_metadata=None)

In [62]:
model.compile(loss=categorical_crossentropy,
              optimizer=Adadelta(),
              metrics=['accuracy'])
model.fit(train, labels,batch_size=batch_size,epochs=epochs,
          verbose=1,validation_split=0.3, callbacks=[tensorboard])
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 23520 samples, validate on 10080 samples
Epoch 1/7
23520/23520 [==============================] - 69s - loss: 1.0216 - acc: 0.5022 - val_loss: 1.0373 - val_acc: 0.4980
Epoch 2/7
23520/23520 [==============================] - 68s - loss: 0.9994 - acc: 0.5018 - val_loss: 1.0197 - val_acc: 0.4983
Epoch 3/7
23520/23520 [==============================] - 67s - loss: 0.9803 - acc: 0.5035 - val_loss: 1.0062 - val_acc: 0.4998
Epoch 4/7
23520/23520 [==============================] - 63s - loss: 0.9652 - acc: 0.5038 - val_loss: 0.9939 - val_acc: 0.5046
Epoch 5/7
23520/23520 [==============================] - 62s - loss: 0.9523 - acc: 0.5071 - val_loss: 0.9820 - val_acc: 0.4959
Epoch 6/7
23520/23520 [==============================] - 61s - loss: 0.9417 - acc: 0.5054 - val_loss: 0.9765 - val_acc: 0.5076
Epoch 7/7
23520/23520 [==============================] - 64s - loss: 0.9301 - acc: 0.5071 - val_loss: 0.9670 - val_acc: 0.5029
Test loss: 0.957139338312
Test accuracy: 0.509880952381
